In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv("/content/flightPrice.csv")

In [3]:
df

,Date_of_journey,Journey_day,Airline,Flight_code,Class,Source,Departure,Total_stops,Arrival,Destination,Duration_in_hours,Days_left,Fare
0,2023-01-16,Monday,SpiceJet,SG-8169,Economy,Delhi,After 6 PM,non-stop,After 6 PM,Mumbai,2.0833,1,5335
1,2023-01-16,Monday,Indigo,6E-2519,Economy,Delhi,After 6 PM,non-stop,Before 6 AM,Mumbai,2.3333,1,5899
2,2023-01-16,Monday,GO FIRST,G8-354,Economy,Delhi,After 6 PM,non-stop,Before 6 AM,Mumbai,2.1667,1,5801
3,2023-01-16,Monday,SpiceJet,SG-8709,Economy,Delhi,After 6 PM,non-stop,After 6 PM,Mumbai,2.0833,1,5794
4,2023-01-16,Monday,Air India,AI-805,Economy,Delhi,After 6 PM,non-stop,After 6 PM,Mumbai,2.1667,1,5955
...,...,...,...,...,...,...,...,...,...,...,...,...,...
452083,2023-03-06,Monday,Vistara,UK-926,Business,Ahmedabad,6 AM - 12 PM,1-stop,After 6 PM,Chennai,13.0833,50,65028
452084,2023-03-06,Monday,Vistara,UK-918,Business,Ahmedabad,Before 6 AM,1-stop,12 PM - 6 PM,Chennai,11.2500,50,69254
452085,2023-03-06,Monday,Vistara,UK-918,Business,Ahmedabad,Before 6 AM,1-stop,12 PM - 6 PM,Chennai,11.2500,50,69254
452086,2023-03-06,Monday,Vistara,UK-946,Business,Ahmedabad,6 AM - 12 PM,1-stop,After 6 PM,Chennai,11.1667,50,72980


In [4]:
df.isnull().sum()

Date_of_journey      0
Journey_day          0
Airline              0
Flight_code          0
Class                0
Source               0
Departure            0
Total_stops          0
Arrival              0
Destination          0
Duration_in_hours    0
Days_left            0
Fare                 0
dtype: int64

In [5]:
df['Date_of_journey'] = pd.to_datetime(df['Date_of_journey'], format='%Y-%m-%d')
df["Day"] = df["Date_of_journey"].dt.day
df["Month"] = df["Date_of_journey"].dt.month
df["Year"] = df["Date_of_journey"].dt.year
df.drop(columns=["Date_of_journey"], inplace=True)

In [6]:
encoder = OneHotEncoder(drop='first', sparse_output=False)

In [7]:
columns_to_encode = ['Class', 'Source','Airline','Journey_day','Destination']

In [8]:
one_hot_encoded_data = encoder.fit_transform(df[columns_to_encode])

In [9]:
one_hot_encoded_df = pd.DataFrame(one_hot_encoded_data, columns=encoder.get_feature_names_out(columns_to_encode))

one_hot_encoded_df.index = df.index

df.drop(columns=columns_to_encode, inplace=True)

df = pd.concat([df, one_hot_encoded_df], axis=1)


In [10]:
label_encoder = LabelEncoder()
df["Total_stops_encoded"] = label_encoder.fit_transform(df["Total_stops"])

df["Total_stops"] = df["Total_stops_encoded"]

In [11]:
df.drop(columns=["Flight_code"], inplace=True)

In [12]:
df["Departure"].unique()

array(['After 6 PM', 'Before 6 AM', '12 PM - 6 PM', '6 AM - 12 PM'],
      dtype=object)

In [13]:
df["Arrival"].unique()

array(['After 6 PM', 'Before 6 AM', '6 AM - 12 PM', '12 PM - 6 PM'],
      dtype=object)

In [14]:
Time_Slots = ['After 6 PM', 'Before 6 AM', '12 PM - 6 PM', '6 AM - 12 PM']

In [15]:
encoder = OneHotEncoder(categories=[Time_Slots,Time_Slots], drop='first', sparse_output=False)

In [16]:
one_hot_encoded_data = encoder.fit_transform(df[['Arrival', 'Departure']])

feature_names = encoder.get_feature_names_out(['Arrival', 'Departure'])

In [17]:
one_hot_encoded_df = pd.DataFrame(one_hot_encoded_data, columns=feature_names)
one_hot_encoded_df.index = df.index

In [18]:
df.drop(columns=['Arrival', 'Departure'], inplace=True)

In [19]:
df = pd.concat([df, one_hot_encoded_df], axis=1)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452088 entries, 0 to 452087
Data columns (total 43 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Total_stops             452088 non-null  int64  
 1   Duration_in_hours       452088 non-null  float64
 2   Days_left               452088 non-null  int64  
 3   Fare                    452088 non-null  int64  
 4   Day                     452088 non-null  int32  
 5   Month                   452088 non-null  int32  
 6   Year                    452088 non-null  int32  
 7   Class_Economy           452088 non-null  float64
 8   Class_First             452088 non-null  float64
 9   Class_Premium Economy   452088 non-null  float64
 10  Source_Bangalore        452088 non-null  float64
 11  Source_Chennai          452088 non-null  float64
 12  Source_Delhi            452088 non-null  float64
 13  Source_Hyderabad        452088 non-null  float64
 14  Source_Kolkata      

In [22]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [23]:
y = df['Fare']
X = df.drop(columns=['Fare'])

In [24]:
X = sm.add_constant(X)

In [25]:
model = sm.OLS(y, X).fit()

In [27]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   Fare   R-squared:                       0.850
Model:                            OLS   Adj. R-squared:                  0.850
Method:                 Least Squares   F-statistic:                 6.385e+04
Date:                Wed, 15 May 2024   Prob (F-statistic):               0.00
Time:                        19:47:32   Log-Likelihood:            -4.6974e+06
No. Observations:              452088   AIC:                         9.395e+06
Df Residuals:                  452047   BIC:                         9.395e+06
Df Model:                          40                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Total_stops            -1661

In [ ]:
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data)

/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


                   Feature          VIF
0              Total_stops          inf
1        Duration_in_hours     1.546832
2                Days_left   440.772500
3                      Day   180.108547
4                    Month   762.605719
5                     Year  4366.172016
6            Class_Economy     1.621059
7              Class_First     1.009719
8    Class_Premium Economy     1.518138
9            Source_Mumbai     2.039708
10         Airline_AirAsia     1.211347
11        Airline_AkasaAir     1.055716
12     Airline_AllianceAir     1.010377
13        Airline_GO FIRST     1.184740
14          Airline_Indigo     1.671787
15        Airline_SpiceJet     1.079709
16         Airline_Vistara     1.693823
17      Journey_day_Monday     1.844968
18    Journey_day_Saturday     1.759478
19      Journey_day_Sunday     1.738156
20    Journey_day_Thursday     1.737067
21     Journey_day_Tuesday     1.797490
22   Journey_day_Wednesday     1.747571
23   Destination_Bangalore     3.872043
